# Creating a Standard Model on Verta

Within Verta, a "Model" can be any arbitrary function: a traditional ML model (e.g., sklearn, PyTorch, TF, etc); a function (e.g., squaring a number, making a DB function etc.); or a mixture of the above (e.g., pre-processing code, a DB call, and then a model application.) See more [here](https://docs.verta.ai/verta/collaboration/concepts).

This notebook provides an example of how to define a a Verta Standard Model by extending [VertaModelBase](https://verta.readthedocs.io/en/master/_autogen/verta.registry.VertaModelBase.html?highlight=VertaModelBase#verta.registry.VertaModelBase). Verta also offers convenience functions for a number of common libraries including scikit-learn, PyTorch, Tensorflow as documented [here](https://verta.readthedocs.io/en/master/_autogen/verta.registry.entities.RegisteredModel.html).

## 0. Imports

In [1]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

In [2]:
# Ensure credentials are set up, if not, use below
#import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST'] = 

from verta import Client

client = Client(os.environ['VERTA_HOST'])

## 1. Register a model

In [3]:
registered_model = client.get_or_create_registered_model(
    name="census", labels=["research-purpose", "team-a"])

In [4]:
from verta.registry import VertaModelBase

class MyModel(VertaModelBase):
    def __init__(self, artifacts):
        self.weights = json.load(open(artifacts["weights"]))
    
    def predict(self, input):
        res = []
        for row in input:
            res.append(row[0] * self.weights[0] + row[1] * self.weights[1])
        return res

In [5]:
from verta.environment import Python

model_version = registered_model.create_standard_model(
    model_cls=MyModel,
    artifacts = {"weights" : [5, 6]},
    environment=Python(requirements=["json"]),
    name="v0",
    labels=["prototype"],
)

## 2. Now you can use the registered model for deployment, monitoring, etc.

---